In [1]:
# SPFDM Assignment 2
# Luuk Oudshoorn and Willem-Jan de Voogd
# Fall 2020

In [5]:
import time
import numpy as np
import pandas as pd
import datetime as dt
from glob import glob
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline
from scipy import interpolate
from scipy.stats import norm
from joblib import Parallel, delayed
import networkx as nx
from scipy.stats import norm
import scipy as sp
import matplotlib.pyplot as plt
import ffn 
from matplotlib import collections as matcoll
%matplotlib inline

Exercise 10 (Stochastic  volatility).
Simulate the paths to maturity T= 1 year. Use the SABR model.
1.  Plot a single path and look if you can observe a volatility clustering.
2.  Plot  histograms  of  log-returns  and  compare  them  to  the  normal  
distribution with the same mean and variance.  Observe the skewness and fattails.

In [ ]:
class MonteCarlo():
    def __init__(self,Npaths, Nsteps, T, F, sigma, alpha, beta, rho, K = 3300, rf=None):
        """Function initialization like in the binomial tree model. 
           Predominantly fixed values"""
        """F is the initial forward rate, sigma the initial stochastic volatility."""
        # If we do not call the function with a value, 
        # initialize at the "standard" numbers
        if rf == 0:
            self.rf = rf
        elif not rf:
            self.rf = np.log((1+0.005/4)**4)
        else:
            self.rf = rf
        self.S0 = F*np.exp(-self.rf*T)
        self.F0 = F
        self.sigma0 = sigma
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.K     = K
        self.Npaths = Npaths
        self.Nsteps = Nsteps
        self.T = T
        self.dT = T/Nsteps
        print(self.rf)
        
    def propagate(self,ft_1,st_1):
        """Equate the monte carlo iteration as function of location xt"""
        """ft and st are, respectively, the forward rate and the stochastic volatility at time t"""
        
        N = 1 if type(ft_1)==np.float64 else len(ft_1)
        # Generate Correlated Brownian motions
        x = np.random.randn(N)
        e = np.sqrt(1-(self.rho)**2) * np.random.randn(N)
        y = self.rho*x+e
        x = x*np.sqrt(self.dT)
        y = y*np.sqrt(self.dT)
        
        # Simulate standard deviations and forward rates
        st = st_1 + self.alpha*st_1*x
        ft = ft_1 + st*(ft_1**self.beta)*y
        return ft, st

    def simulate(self):
        """Iterate through the MC process for N steps and 1 path"""
        """ft and st are, respectively, the forward rate and the stochastic volatility at time t"""
        ft = np.zeros(self.Nsteps)
        st = np.zeros(self.Nsteps)
        prices = np.zeros(self.Nsteps)
        # Initialize at start value
        ft[0] = self.F0
        st[0] = self.sigma0
        prices[0] = self.S0
        
        for i in range(self.Nsteps-1):
            # propagate prices
            ft[i+1], st[i+1] = self.propagate(ft[i], st[i])
            prices[i+1] = ft[i+1]*np.exp(-self.rf*(self.T - (i+1)*self.dT))
        return prices, st
        
    def plot_single(self, filename=None):
        """Plot a single realization"""
        prices, volatilities = self.simulate()
        timearr = np.arange(0,self.T,self.dT)
        plt.plot(timearr,prices,color='black',lw=0.5)
        # plot horitontal line at long term mean
#         plt.axhline(self.ltm,ls='--',color='black',lw=1)
        plt.xlabel('Time [yr]')
        plt.ylabel('Stock price [USD]')
        plt.tight_layout()
        if filename:
            plt.savefig(filename)
        plt.show()
        plt.plot(volatilities)
        plt.show()
        
    def simulate_many(self):
        """Simulate Npaths for Nsteps"""
        """ft and st are, respectively, the forward rate and the stochastic volatility at time t"""
        ft = np.zeros((self.Npaths, self.Nsteps))
        st = np.zeros((self.Npaths, self.Nsteps))
        prices = np.zeros((self.Npaths, self.Nsteps))
        # Same as before but now a whole matrix with N paths
        ft[:,0] = self.F0
        st[:,0] = self.sigma0
        prices[:,0] = self.S0
        for i in range(self.Nsteps-1):
            ft[:,i+1], st[:,i+1] = self.propagate(ft[:,i], st[:,i])
            prices[:,i+1] = ft[:,i+1]*np.exp(-self.rf*(self.T - (i+1)*self.dT))
        return prices, st
        
    def plot_many(self, logreturns=False):
        """Make a plot of the overall distribution and also plot the resulting PDF of final 
           prices."""
        prices,vola = self.simulate_many()
        if logreturns==True:
            P = np.diff(np.log(prices))
            timearr = np.arange(self.dT,self.T,self.dT)
        else: 
            P = prices
            timearr = np.arange(0,self.T,self.dT)
        fig, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [4, 1]},figsize=(8,1.7))
        for i in range(self.Npaths-1):
            a0.plot(timearr,P[i,:],color='black',lw=0.002)
        a0.plot(timearr,P[i,:],color='green',lw=1.0)
        a0.set_xlabel('Time [yr]')
        a0.set_ylabel('Stock price [USD]')
        
        bins = np.linspace(np.quantile(P[:,-1],0.01),np.quantile(P[:,-1],0.99), 100)
        a1.hist(P[:,-1],bins=bins, color='gray', density=True,orientation='horizontal')
        mean,std=norm.fit(P[:,-1])
        #x = np.linspace(-0.015, 0.015, 100)
        #y = norm.pdf(x, mean, std)
        #a1.plot(x, y)
        a1.set_xticks([])
        a1.set_yticks([])
        a0.set_rasterized(True)
        fig.tight_layout(pad=-0.3)
        a0.set_ylim(np.quantile(P[:,-1],0.01),np.quantile(P[:,-1],0.99))
        a1.set_ylim(np.quantile(P[:,-1],0.01),np.quantile(P[:,-1],0.99))
        plt.savefig('manyMCtracks_PDF.png', dpi=400,bbox_inches='tight')
        plt.show()

        
    def black_scholes(self,S, K, T, r, sigma, call_put='call'):   
        """Return black scholes price of an option"""
        #S: spot price
        #K: strike price
        #T: time to maturity
        #r: interest rate
        #sigma: volatility of underlying asset
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

        if call_put == 'call':
            optionprice = (S * norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * norm.cdf(d2, 0.0, 1.0))
        elif call_put == 'put':
            optionprice = (K * np.exp(-r * T) * norm.cdf(-d2, 0.0, 1.0) - S * norm.cdf(-d1, 0.0, 1.0))
        return optionprice

    
    def minimizer(self,params,type_):
        SABR_price, strike = params
        optim = minimize(fun=(lambda implied_vola: np.abs(self.black_scholes(self.F0, 
                                                     strike, 
                                                     self.T,
                                                     self.rf,
                                                     implied_vola,
                                                     call_put=type_)-SABR_price)),x0=0.4,tol=1e-10)#,method='Nelder-Mead')
        return optim.x

    def parallel_option(self,K,type_='call'):
        return self.option_price(K=K,optiontype=type_)
    
    def calc_implied_vola(self):
        """For different strikes, we calculate the SABR option price and the implied volatility
           which is the volatility to match the BS price to the SABR option price"""
        # Define range of strike ratios
        ratiorange = np.arange(0.3,3,0.015)
        # Convert ratio to true strike price
        strikerange = ratiorange * self.F0
        # for each of these strikes, obtain SABR option price
        prices = np.array(Parallel(n_jobs=22)(delayed(self.parallel_option)(K) for K in strikerange))
        # obtain implied vola for every price
        
        implied_volas = np.zeros(len(prices))
        BSprices      = np.zeros(len(prices))
        
        
        implied_volas = np.array(Parallel(n_jobs=22)(delayed(self.minimizer)(params,type_='call') for params in zip(prices,strikerange)))
        BSprices = np.array([self.black_scholes(self.F0,strikerange[w], 
                                                 self.T,
                                                 self.rf,
                                                 implied_volas[w],
                                                 call_put='call') for w in range(len(prices))])
        for i in zip(prices,strikerange):
            print(i)
        plt.plot(ratiorange, implied_volas*100)
        plt.xlabel(r'$\frac{K}{S_0}$')
        plt.ylabel('Implied volatility (%)')
        plt.tight_layout()
        plt.savefig('Impliedvola_BS.pdf')
        plt.show()
        plt.plot(ratiorange, BSprices,label='BS')
        plt.plot(ratiorange, prices, label='SABR')
        plt.legend()
        plt.show()
        
    def plot_prices_strikes(self):
        """This function will create K/S0 plot of call and put prices for SABR and BS model"""
        # Define range of strike ratios
        ratiorange = np.arange(0.75,1.25,0.01)
        # Convert ratio to true strike price
        strikerange = ratiorange * self.F0
        # for each of these strikes, obtain SABR option price
        SABRcall = np.array(Parallel(n_jobs=22)(delayed(self.parallel_option)(K,'call') for K in strikerange))
        SABRput  = np.array(Parallel(n_jobs=22)(delayed(self.parallel_option)(K,'put') for K in strikerange))
    
        # Set BS vola to initial SABR vola
        # Find volatility that sets BS price at K=S to SABR price
        BSvolacall = self.minimizer((SABRcall[np.argmin(np.abs(ratiorange-1))],self.F0),'call')
        BSvolaput  = self.minimizer((SABRput[np.argmin(np.abs(ratiorange-1))],self.F0),'put')
        
        
        # Obtain BS prices for calls and puts 
        BScall   = np.array([self.black_scholes(self.F0,strike, 
                                                 self.T,
                                                 self.rf,
                                                 BSvolacall,
                                                 call_put='call') for strike in strikerange])
        BSput   = np.array([self.black_scholes(self.F0,strike, 
                                                 self.T,
                                                 self.rf,
                                                 BSvolaput,
                                                 call_put='put') for strike in strikerange])
        
        # Start plotting
        fig, ax = plt.subplots()
        ax.plot(ratiorange,SABRcall, label='SABR Call')
        ax.plot(ratiorange,SABRput,  label='SABR Put')
        ax.plot(ratiorange,BScall,   label='BS Call')
        ax.plot(ratiorange,BSput,    label='BS Put')
        ax.set_xlabel(r'$\frac{K}{S_0}$')
        ax.set_ylabel('Option price')
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    def option_price(self, optiontype='call',K=None):
        """Equate option price by taking option value for each path, averaging and discounting back 
           to present day value using risk free rate. """
        # Get prices 
        if not K:
            K = self.K
        prices, volatility = self.simulate_many()
        # Get final prices
        finalprices = prices[:,-1]
        # calculate option values
        if optiontype == 'call':
            optionvalues = np.max(np.vstack((np.zeros(len(finalprices)),finalprices-K)),axis=0)
        elif optiontype == 'put':
            optionvalues = np.max(np.vstack((np.zeros(len(finalprices)),K-finalprices)),axis=0)
        optionvalue = np.mean(optionvalues)
        # Discount to present value
        optionvalue = np.exp(-self.rf*self.T) * optionvalue
        return optionvalue

MC = MonteCarlo(10000,250,T=1.5,F=3500, sigma=0.2, alpha=0.5, beta=1, rho=-0.7,K=3300, rf=0)
MC.plot_many()

0
